In [83]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [167]:
#url = 'https://www.allrecipes.com/recipes/227/world-cuisine/asian/'
url = 'https://www.allrecipes.com/recipes/723/world-cuisine/european/italian/'
response = requests.get(url)
print(response)

<Response [200]>


In [168]:
soup = BeautifulSoup(response.text, "html.parser")
links = soup.select("a[data-click-id*=cardslot]")

In [169]:
print(links)

[<a data-click-id="cardslot 1" data-content-provider-id="" data-internal-referrer-link="staff pick" href="https://www.allrecipes.com/recipe/259466/easy-caponata/">
<img alt="Easy Caponata Recipe - Caponata is a traditional vegetarian dish from Sicily with eggplant, zucchini, tomato, peppers, pine nuts, and raisins cooked in olive oil." class="fixed-recipe-card__img" data-lazy-load="" data-original-src="https://images.media-allrecipes.com/userphotos/300x300/4548530.jpg" src="https://images.media-allrecipes.com/ar/spacer.gif" style="display: inline;" title="Easy Caponata Recipe"/>
</a>, <a data-click-id="cardslot 2" data-content-provider-id="" data-internal-referrer-link="staff pick" href="https://www.allrecipes.com/recipe/255143/chef-johns-lemon-ice/">
<img alt="Chef John's Lemon Ice Recipe and Video - Using the classic Italian granita technique makes this an easy and refreshing summer treat. There's a delicious contrast between that light texture and bright, vibrant flavor." class="fix

In [170]:
import re
cardslotRegex = r"<a data-click-id=\"cardslot (\d+)"
imagesrcRegex = r'(?<=data-original-src=")([^"]+)(?=")'
tempmatch = ""
recipeLinks = []
recipeImageLinks =  []
for link in links:
    match = re.search(cardslotRegex,str(link)).group(0)
    if match == tempmatch:
        pass
    else:
        tempmatch = match
        recipeLink = link["href"]
        recipeLinks.append(recipeLink)
        recipeImageLink = re.search(imagesrcRegex,str(link)).group(0)
        recipeImageLinks.append(recipeImageLink)

In [171]:
for recipeLink in recipeLinks:
    print(recipeLink)
    response = requests.get(recipeLink)
    print(response)
    soup = BeautifulSoup(response.text, "html.parser")
    try:
        title = soup.select("h1[class*=heading-content]")[0].text.strip()
    except:
        title = soup.select("h1")[0].text.strip()
    print(title)
    ingredients = soup.select("label[ng-class*=true:]")
    for item in ingredients:
        ingredientRegex = r'(?<=title=")([^"]+)(?=")'
        try:
            ingredient = re.search(ingredientRegex,str(item)).group(0)
            print(ingredient)
        except:
            pass
    ingredients = soup.select("span[class*=ingredients-item-name]")
    for item in ingredients:
        ingredient = item.text.strip()
        print(ingredient)    
print("done")

https://www.allrecipes.com/recipe/259466/easy-caponata/
<Response [200]>
Easy Caponata
2 stalks stalk celery, chopped into 1/4-inch pieces
1/2 cup olive oil
1 onion, thinly sliced
1 medium eggplant, chopped into bite-size pieces
2 red bell peppers, thinly sliced
2 zucchini, cubed
1 3/4 cups tomato puree (passata)
1/2 cup white vinegar
1 tablespoon white sugar
10 black olives, pitted
2 tablespoons raisins
2 tablespoons pine nuts
1 teaspoon capers
salt and ground black pepper to taste
10 leaves basil, chopped
https://www.allrecipes.com/recipe/255143/chef-johns-lemon-ice/
<Response [200]>
Chef John's Lemon Ice
4 cups cold water
7/8 cup white sugar
2 tablespoons lemon zest
1 cup freshly squeezed lemon juice
1 teaspoon lemon extract
https://www.allrecipes.com/recipe/235857/focaccia-di-recco/
<Response [200]>
Focaccia di Recco
2 cups all-purpose flour
1/2 cup cold water
2 tablespoons cold water
1 tablespoon extra-virgin olive oil
3/4 teaspoon fine salt
1 quarter-sheet baking pan
1 teaspoon e

<Response [200]>
Chicken Milano
1 tablespoon butter
2 cloves garlic, minced
½ cup sun-dried tomatoes, chopped
1 cup chicken broth, divided
1 cup heavy cream
1 pound skinless, boneless chicken breast halves
salt and pepper to taste
2 tablespoons vegetable oil
2 tablespoons chopped fresh basil
8 ounces dry fettuccini pasta
https://www.allrecipes.com/recipe/11729/american-lasagna/
<Response [200]>
American Lasagna
1 1/2 pounds lean ground beef
1 onion, chopped
2 cloves garlic, minced
1 tablespoon chopped fresh basil
1 teaspoon dried oregano
2 tablespoons brown sugar
1 1/2 teaspoons salt
1 (29 ounce) can diced tomatoes
2 (6 ounce) cans tomato paste
12 dry lasagna noodles
2 eggs, beaten
1 pint part-skim ricotta cheese
1/2 cup grated Parmesan cheese
2 tablespoons dried parsley
1 teaspoon salt
1 pound mozzarella cheese, shredded
2 tablespoons grated Parmesan cheese
https://www.allrecipes.com/recipe/16427/slow-cooker-italian-beef-for-sandwiches/
<Response [200]>
Slow Cooker Italian Beef for Sa

In [165]:
#Get Recipe Title
print(recipeLinks[2])
response = requests.get(recipeLinks[2])
print(response)
soup = BeautifulSoup(response.text, "html.parser")
try:
    title = soup.select("h1[class*=heading-content]")[1].text.strip()
except:
    title = soup.select("h1")[0].text.strip()
    print(title)

#Get Ingredients 
ingredients = soup.select("label[ng-class*=true:]")
for item in ingredients:
    ingredientRegex = r'(?<=title=")([^"]+)(?=")'
    try:
        ingredient = re.search(ingredientRegex,str(item)).group(0)
        print(ingredient)
    except:
        pass
ingredients = soup.select("span[class*=ingredients-item-name]")
for item in ingredients:
    ingredient = item.text.strip()
    print(ingredient)



https://www.allrecipes.com/recipe/239818/iranian-persian-salad-shirazi/
<Response [200]>
Iranian / Persian Salad Shirazi
4 ripe tomatoes, seeded and diced
1 red onion, diced
2 small English cucumbers, peeled and diced
3 tablespoons olive oil
lemon, juiced
1 tablespoon dried mint
1 teaspoon dried dill
1 teaspoon ground sumac
salt and ground black pepper to taste
